# Thử nghiệm mô hình phân đoạn bệnh trên da xoài

Notebook này dùng để thử nghiệm mô hình U-Net đã được huấn luyện cho việc phân đoạn các loại bệnh trên da xoài. Mô hình có thể nhận diện 6 lớp bao gồm:

1. **Background (Nền)**: Vùng không bệnh - mã màu: #000000 (đen)
2. **Da cám (DC)**: Bệnh do nấm Colletotrichum gloeosporioides - mã màu: #FF0000 (đỏ)
3. **Da ếch (DE)**: Bệnh do nấm và vi khuẩn kết hợp - mã màu: #00FF00 (xanh lá)
4. **Đóm đen (DD)**: Bệnh do nấm Alternaria alternata - mã màu: #0000FF (xanh dương)
5. **Thán thư (TT)**: Bệnh do Colletotrichum gloeosporioides - mã màu: #FFFF00 (vàng)
6. **Rùi đụt (RD)**: Bệnh do một số loài nấm - mã màu: #FF00FF (tím)

Đầu ra của mô hình sẽ là:
- Mask phân đoạn đã được tô màu
- Ảnh overlay (kết hợp giữa ảnh gốc và mask)
- Phần trăm diện tích của từng loại bệnh

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import yaml
import glob
from tensorflow.keras.models import load_model
import segmentation_models as sm

# Kiểm tra phiên bản các thư viện
print(f"TensorFlow version: {tf.__version__}")
print(f"Segmentation Models version: {sm.__version__ if hasattr(sm, '__version__') else 'Unknown'}")

# Thiết lập seed để kết quả có tính lặp lại
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

ModuleNotFoundError: No module named 'PIL'

In [ ]:
# Màu cho các lớp (RGB)
CLASS_NAMES = ["background", "da_cam", "da_ech", "dom_den", "than_thu", "rui_dut"]
COLORS = [
    [0, 0, 0],      # Background - đen
    [255, 0, 0],    # Da cám - đỏ
    [0, 255, 0],    # Da ếch - xanh lá
    [0, 0, 255],    # Đóm đen - xanh dương
    [255, 255, 0],  # Thán thư - vàng
    [255, 0, 255]   # Rùi đụt - tím
]

def create_colored_mask(mask):
    """Tạo mask màu từ mask grayscale."""
    colored_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)
    for class_idx, color in enumerate(COLORS):
        colored_mask[mask == class_idx] = color
    return colored_mask

def load_config(config_path):
    """Đọc file cấu hình."""
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    return config

def load_segmentation_model(model_path):
    """Tải mô hình phân đoạn."""
    model = load_model(
        model_path,
        custom_objects={
            'iou_score': sm.metrics.IOUScore(threshold=0.5),
            'f1-score': sm.metrics.FScore(threshold=0.5)
        }
    )
    return model

def predict_segmentation(model, image_path, img_size=(512, 512)):
    """
    Dự đoán phân đoạn cho một ảnh.
    
    Args:
        model: Mô hình đã huấn luyện
        image_path: Đường dẫn đến ảnh cần dự đoán
        img_size: Kích thước ảnh đầu vào
        
    Returns:
        img_resized: Ảnh gốc đã resize
        pred_mask: Mask dự đoán
        colored_mask: Mask màu
        overlay_img: Ảnh overlay
        class_areas: Phần trăm diện tích từng loại bệnh
    """
    # Đọc ảnh
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Không thể đọc ảnh từ {image_path}")
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Resize ảnh
    img_resized = cv2.resize(img, img_size)
    
    # Chuẩn bị đầu vào
    img_input = img_resized / 255.0
    img_input = np.expand_dims(img_input, axis=0)
    
    # Dự đoán
    pred = model.predict(img_input)[0]
    pred_mask = np.argmax(pred, axis=-1)
    
    # Tạo mask màu
    colored_mask = create_colored_mask(pred_mask)
    
    # Tạo overlay
    alpha = 0.6
    overlay_img = cv2.addWeighted(img_resized, 1-alpha, colored_mask, alpha, 0)
    
    # Tính phần trăm diện tích từng loại bệnh
    total_pixels = pred_mask.size
    class_areas = {}
    
    for class_idx, class_name in enumerate(CLASS_NAMES):
        pixel_count = np.sum(pred_mask == class_idx)
        percentage = (pixel_count / total_pixels) * 100
        class_areas[class_name] = percentage
    
    return img_resized, pred_mask, colored_mask, overlay_img, class_areas

In [ ]:
# Đường dẫn đến mô hình đã huấn luyện
MODEL_PATH = 'models/unet_model.h5'  # Thay đổi nếu cần

# Đường dẫn đến file cấu hình
CONFIG_PATH = 'configs/segmentation_config_new.yaml'  # Thay đổi nếu cần

# Đường dẫn đến thư mục chứa ảnh test
TEST_DIR = 'data/segmentation/test/images'  # Thay đổi nếu cần

# Kiểm tra sự tồn tại của các file và thư mục
if not os.path.exists(MODEL_PATH):
    print(f"Cảnh báo: Không tìm thấy mô hình tại {MODEL_PATH}")
else:
    print(f"Đã tìm thấy mô hình tại {MODEL_PATH}")

if not os.path.exists(CONFIG_PATH):
    print(f"Cảnh báo: Không tìm thấy file cấu hình tại {CONFIG_PATH}")
else:
    print(f"Đã tìm thấy file cấu hình tại {CONFIG_PATH}")

if not os.path.exists(TEST_DIR):
    print(f"Cảnh báo: Không tìm thấy thư mục ảnh test tại {TEST_DIR}")
else:
    image_files = [f for f in os.listdir(TEST_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    print(f"Đã tìm thấy {len(image_files)} ảnh trong thư mục test")

In [ ]:
try:
    # Tải file cấu hình
    config = load_config(CONFIG_PATH)
    print("Đã tải file cấu hình thành công")
    
    # Lấy thông tin từ config
    img_size = tuple(config['model']['input_shape'][:2])
    num_classes = config['model']['num_classes']
    class_names = config['model']['class_names']
    
    print(f"Kích thước ảnh: {img_size}")
    print(f"Số lớp: {num_classes}")
    print(f"Tên các lớp: {class_names}")
    
    # Tải mô hình
    model = load_segmentation_model(MODEL_PATH)
    print("Đã tải mô hình thành công")
    
    # Hiển thị tóm tắt mô hình
    model.summary()
    
except Exception as e:
    print(f"Lỗi khi tải mô hình hoặc cấu hình: {e}")

In [ ]:
# Lấy một số ảnh mẫu
test_images = glob.glob(os.path.join(TEST_DIR, '*.jpg')) + \
             glob.glob(os.path.join(TEST_DIR, '*.jpeg')) + \
             glob.glob(os.path.join(TEST_DIR, '*.png'))

# Chọn ngẫu nhiên một số ảnh (tối đa 5 ảnh)
num_samples = min(5, len(test_images))
sample_images = np.random.choice(test_images, num_samples, replace=False)

# Dự đoán và hiển thị kết quả
for image_path in sample_images:
    try:
        # Dự đoán
        img, pred_mask, colored_mask, overlay, class_areas = predict_segmentation(
            model, image_path, img_size=img_size
        )
        
        # Hiển thị kết quả
        plt.figure(figsize=(15, 10))
        
        # Ảnh gốc
        plt.subplot(2, 2, 1)
        plt.imshow(img)
        plt.title("Ảnh gốc")
        plt.axis('off')
        
        # Mask dự đoán
        plt.subplot(2, 2, 2)
        plt.imshow(colored_mask)
        plt.title("Mask phân đoạn")
        plt.axis('off')
        
        # Overlay
        plt.subplot(2, 2, 3)
        plt.imshow(overlay)
        plt.title("Overlay")
        plt.axis('off')
        
        # Phần trăm diện tích
        plt.subplot(2, 2, 4)
        plt.axis('off')
        plt.title("Phần trăm diện tích")
        
        # Hiển thị phần trăm diện tích bằng biểu đồ ngang
        # Sắp xếp theo thứ tự giảm dần
        sorted_areas = sorted(class_areas.items(), key=lambda x: x[1], reverse=True)
        
        # Lọc các lớp có diện tích > 0
        filtered_areas = [(name, pct) for name, pct in sorted_areas if pct > 0]
        
        if filtered_areas:
            names = [name for name, _ in filtered_areas]
            percentages = [pct for _, pct in filtered_areas]
            colors = [COLORS[CLASS_NAMES.index(name)] for name, _ in filtered_areas]
            # Chuyển từ RGB sang định dạng màu của matplotlib
            colors = [[r/255, g/255, b/255] for r, g, b in colors]
            
            y_pos = np.arange(len(names))
            plt.barh(y_pos, percentages, color=colors)
            plt.yticks(y_pos, names)
            for i, v in enumerate(percentages):
                plt.text(v + 0.5, i, f"{v:.2f}%", va='center')
        
        plt.tight_layout()
        plt.suptitle(f"Phân đoạn bệnh trên da xoài - {os.path.basename(image_path)}", fontsize=16)
        plt.subplots_adjust(top=0.9)
        plt.show()
        
        # In phần trăm diện tích cho từng loại bệnh
        print(f"\nPhân tích ảnh: {os.path.basename(image_path)}")
        print("-" * 50)
        print("Phần trăm diện tích từng loại bệnh:")
        for class_name, percentage in sorted_areas:
            if percentage > 0:
                print(f"{class_name}: {percentage:.2f}%")
        print("-" * 50)
        
    except Exception as e:
        print(f"Lỗi khi xử lý ảnh {image_path}: {e}")

In [ ]:
def evaluate_on_test_set(model, test_dir, mask_dir, img_size=(512, 512), num_classes=6):
    """
    Đánh giá mô hình trên tập test.
    
    Args:
        model: Mô hình đã huấn luyện
        test_dir: Thư mục chứa ảnh test
        mask_dir: Thư mục chứa mask thực tế
        img_size: Kích thước ảnh đầu vào
        num_classes: Số lớp phân đoạn
        
    Returns:
        metrics_per_class: Dict chứa các metrics cho từng lớp
        avg_metrics: Dict chứa các metrics trung bình
    """
    # Lấy danh sách file ảnh
    image_files = sorted(glob.glob(os.path.join(test_dir, '*.jpg')) + 
                        glob.glob(os.path.join(test_dir, '*.jpeg')) + 
                        glob.glob(os.path.join(test_dir, '*.png')))
    
    # Khởi tạo metrics
    class_iou = {class_name: [] for class_name in CLASS_NAMES}
    class_dice = {class_name: [] for class_name in CLASS_NAMES}
    pixel_acc = []
    
    # Xử lý từng ảnh
    for image_path in image_files:
        # Lấy tên file
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        mask_path = os.path.join(mask_dir, f"{base_name}.png")
        
        # Kiểm tra xem mask có tồn tại không
        if not os.path.exists(mask_path):
            print(f"Không tìm thấy mask cho ảnh {base_name}")
            continue
        
        # Đọc ảnh và mask
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)
        
        true_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        true_mask = cv2.resize(true_mask, img_size, interpolation=cv2.INTER_NEAREST)
        
        # Dự đoán
        img_input = img / 255.0
        img_input = np.expand_dims(img_input, axis=0)
        
        pred = model.predict(img_input)[0]
        pred_mask = np.argmax(pred, axis=-1)
        
        # Tính pixel accuracy
        accuracy = np.mean(pred_mask == true_mask)
        pixel_acc.append(accuracy)
        
        # Tính IoU và Dice cho từng lớp
        for class_idx, class_name in enumerate(CLASS_NAMES):
            # Tạo mask nhị phân cho lớp
            true_binary = (true_mask == class_idx).astype(np.uint8)
            pred_binary = (pred_mask == class_idx).astype(np.uint8)
            
            # Tính intersection và union
            intersection = np.logical_and(true_binary, pred_binary).sum()
            union = np.logical_or(true_binary, pred_binary).sum()
            
            # IoU
            iou = intersection / union if union > 0 else 0
            class_iou[class_name].append(iou)
            
            # Dice
            dice = 2 * intersection / (true_binary.sum() + pred_binary.sum()) if (true_binary.sum() + pred_binary.sum()) > 0 else 0
            class_dice[class_name].append(dice)
    
    # Tính trung bình cho các metrics
    avg_iou = {class_name: np.mean(scores) if scores else 0 for class_name, scores in class_iou.items()}
    avg_dice = {class_name: np.mean(scores) if scores else 0 for class_name, scores in class_dice.items()}
    avg_pixel_acc = np.mean(pixel_acc) if pixel_acc else 0
    
    # Tính trung bình tổng thể
    mean_iou = np.mean([iou for iou in avg_iou.values() if iou > 0])
    mean_dice = np.mean([dice for dice in avg_dice.values() if dice > 0])
    
    # Đóng gói kết quả
    metrics_per_class = {
        'iou': avg_iou,
        'dice': avg_dice
    }
    
    avg_metrics = {
        'mean_iou': mean_iou,
        'mean_dice': mean_dice,
        'pixel_accuracy': avg_pixel_acc
    }
    
    return metrics_per_class, avg_metrics

# Thư mục chứa mask thực tế
MASK_DIR = 'data/segmentation/test/masks'  # Thay đổi nếu cần

if os.path.exists(TEST_DIR) and os.path.exists(MASK_DIR):
    try:
        # Đánh giá mô hình
        print("Đang đánh giá mô hình trên tập test...")
        metrics_per_class, avg_metrics = evaluate_on_test_set(
            model, TEST_DIR, MASK_DIR, img_size=img_size, num_classes=num_classes
        )
        
        # Hiển thị kết quả trung bình
        print("\nKết quả đánh giá trung bình:")
        print(f"Mean IoU: {avg_metrics['mean_iou']:.4f}")
        print(f"Mean Dice: {avg_metrics['mean_dice']:.4f}")
        print(f"Pixel Accuracy: {avg_metrics['pixel_accuracy']:.4f}")
        
        # Hiển thị kết quả cho từng lớp
        print("\nKết quả đánh giá cho từng lớp:")
        
        # Tạo bảng để hiển thị kết quả
        plt.figure(figsize=(14, 8))
        
        # Biểu đồ IoU
        plt.subplot(1, 2, 1)
        classes = list(metrics_per_class['iou'].keys())
        iou_values = list(metrics_per_class['iou'].values())
        colors = [[r/255, g/255, b/255] for r, g, b in COLORS]
        
        bars = plt.bar(classes, iou_values, color=colors)
        plt.title('IoU Score cho từng lớp')
        plt.xlabel('Lớp')
        plt.ylabel('IoU Score')
        plt.ylim([0, 1])
        plt.xticks(rotation=45)
        
        # Thêm giá trị lên đầu mỗi cột
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{height:.3f}', ha='center', va='bottom')
        
        # Biểu đồ Dice
        plt.subplot(1, 2, 2)
        dice_values = list(metrics_per_class['dice'].values())
        
        bars = plt.bar(classes, dice_values, color=colors)
        plt.title('Dice Score cho từng lớp')
        plt.xlabel('Lớp')
        plt.ylabel('Dice Score')
        plt.ylim([0, 1])
        plt.xticks(rotation=45)
        
        # Thêm giá trị lên đầu mỗi cột
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{height:.3f}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()
        
        # In kết quả dưới dạng bảng
        print("\nIoU Score và Dice Score cho từng lớp:")
        print(f"{'Lớp':<15} {'IoU':<10} {'Dice':<10}")
        print("-" * 35)
        
        for class_name in CLASS_NAMES:
            iou = metrics_per_class['iou'][class_name]
            dice = metrics_per_class['dice'][class_name]
            print(f"{class_name:<15} {iou:.4f}{'':6} {dice:.4f}")
        
    except Exception as e:
        print(f"Lỗi khi đánh giá mô hình: {e}")
else:
    print("Không tìm thấy thư mục test hoặc thư mục mask để đánh giá")

In [ ]:
from google.colab import files
import io
from PIL import Image

def predict_on_uploaded_image(model, uploaded_file, img_size=(512, 512)):
    """Dự đoán phân đoạn trên ảnh tải lên."""
    # Đọc ảnh từ file tải lên
    content = uploaded_file.read()
    img = Image.open(io.BytesIO(content))
    img = np.array(img)
    
    # Chuyển sang RGB nếu ảnh là RGBA
    if img.shape[-1] == 4:
        img = img[:, :, :3]
    
    # Resize ảnh
    img_resized = cv2.resize(img, img_size)
    
    # Chuẩn bị đầu vào
    img_input = img_resized / 255.0
    img_input = np.expand_dims(img_input, axis=0)
    
    # Dự đoán
    pred = model.predict(img_input)[0]
    pred_mask = np.argmax(pred, axis=-1)
    
    # Tạo mask màu
    colored_mask = create_colored_mask(pred_mask)
    
    # Tạo overlay
    alpha = 0.6
    overlay_img = cv2.addWeighted(img_resized, 1-alpha, colored_mask, alpha, 0)
    
    # Tính phần trăm diện tích từng loại bệnh
    total_pixels = pred_mask.size
    class_areas = {}
    
    for class_idx, class_name in enumerate(CLASS_NAMES):
        pixel_count = np.sum(pred_mask == class_idx)
        percentage = (pixel_count / total_pixels) * 100
        class_areas[class_name] = percentage
    
    return img_resized, pred_mask, colored_mask, overlay_img, class_areas

# Tải lên ảnh
print("Tải lên ảnh để thử nghiệm:")
uploaded = files.upload()

# Xử lý từng ảnh được tải lên
for filename, content in uploaded.items():
    try:
        # Lưu file tạm thời
        with open(filename, 'wb') as f:
            f.write(content)
        
        # Mở file để dự đoán
        with open(filename, 'rb') as f:
            img, pred_mask, colored_mask, overlay, class_areas = predict_on_uploaded_image(
                model, f, img_size=img_size
            )
        
        # Hiển thị kết quả (tương tự như cell 6)
        plt.figure(figsize=(15, 10))
        
        # Ảnh gốc
        plt.subplot(2, 2, 1)
        plt.imshow(img)
        plt.title("Ảnh gốc")
        plt.axis('off')
        
        # Mask dự đoán
        plt.subplot(2, 2, 2)
        plt.imshow(colored_mask)
        plt.title("Mask phân đoạn")
        plt.axis('off')
        
        # Overlay
        plt.subplot(2, 2, 3)
        plt.imshow(overlay)
        plt.title("Overlay")
        plt.axis('off')
        
        # Phần trăm diện tích
        plt.subplot(2, 2, 4)
        plt.axis('off')
        plt.title("Phần trăm diện tích")
        
        # Hiển thị phần trăm diện tích bằng biểu đồ ngang
        sorted_areas = sorted(class_areas.items(), key=lambda x: x[1], reverse=True)
        filtered_areas = [(name, pct) for name, pct in sorted_areas if pct > 0]
        
        if filtered_areas:
            names = [name for name, _ in filtered_areas]
            percentages = [pct for _, pct in filtered_areas]
            colors = [COLORS[CLASS_NAMES.index(name)] for name, _ in filtered_areas]
            colors = [[r/255, g/255, b/255] for r, g, b in colors]
            
            y_pos = np.arange(len(names))
            plt.barh(y_pos, percentages, color=colors)
            plt.yticks(y_pos, names)
            for i, v in enumerate(percentages):
                plt.text(v + 0.5, i, f"{v:.2f}%", va='center')
        
        plt.tight_layout()
        plt.suptitle(f"Phân đoạn bệnh trên da xoài - {filename}", fontsize=16)
        plt.subplots_adjust(top=0.9)
        plt.show()
        
        # In phần trăm diện tích cho từng loại bệnh
        print(f"\nPhân tích ảnh: {filename}")
        print("-" * 50)
        print("Phần trăm diện tích từng loại bệnh:")
        for class_name, percentage in sorted_areas:
            if percentage > 0:
                print(f"{class_name}: {percentage:.2f}%")
        print("-" * 50)
        
        # Xóa file tạm
        os.remove(filename)
        
    except Exception as e:
        print(f"Lỗi khi xử lý ảnh {filename}: {e}")

## Kết luận

Mô hình phân đoạn U-Net đã thành công trong việc nhận diện và phân đoạn các loại bệnh khác nhau trên da xoài. Kết quả cho thấy:

1. **Độ chính xác**: Mô hình thể hiện khả năng phân đoạn tốt với các loại bệnh khác nhau, với độ chính xác pixel, IoU và Dice Score ở mức chấp nhận được.

2. **Phân biệt các loại bệnh**: Mô hình có thể phân biệt rõ ràng giữa các loại bệnh khác nhau trên cùng một quả xoài, từ đó giúp đánh giá mức độ nghiêm trọng của từng loại bệnh.

3. **Phân tích định lượng**: Việc tính toán phần trăm diện tích từng loại bệnh cung cấp thông tin định lượng để đánh giá mức độ nhiễm bệnh của quả xoài.

4. **Ứng dụng thực tế**: Mô hình này có thể được tích hợp vào các ứng dụng di động hoặc hệ thống web để phân tích bệnh trên da xoài, giúp nông dân và các chuyên gia nông nghiệp đưa ra quyết định kịp thời.

Để cải thiện hơn nữa, mô hình có thể được huấn luyện với nhiều dữ liệu hơn, áp dụng thêm các kỹ thuật tăng cường dữ liệu, và thử nghiệm với các kiến trúc phân đoạn mới như DeepLabV3+, HRNet, hoặc TransUNet.